In [1]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Loading Dataset
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
#checking counts-Rows&columns
dataset.shape

(400, 5)

In [4]:
#get dummies
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [5]:
#dropping the column that is not required which has no information
dataset=dataset.drop("User ID",axis=1)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [6]:
#How many classes we are going to classify
dataset["Purchased"].value_counts()


Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
#spliting Independent and Dependent
Independent=dataset[["Age","EstimatedSalary","Gender_Male"]]
Dependent=dataset["Purchased"]

In [8]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Independent,Dependent, test_size = 1/3, random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [11]:
#model creation
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha':[1],'force_alpha':[True], 'fit_prior':[True], 'class_prior':[None]} 
grid = GridSearchCV(CategoricalNB(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 
        

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Users\sswat\AppData\Local\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [1], 'class_prior': [None],
                         'fit_prior': [True], 'force_alpha': [True]},
             scoring='f1_weighted', verbose=3)

In [14]:
grid_predictions = grid.predict(X_test)
        

In [15]:
#evaluation metrics(confusion_matrix)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
        

In [16]:
print(cm)

[[82  3]
 [18 31]]


In [17]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [18]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.82      0.96      0.89        85
           1       0.91      0.63      0.75        49

    accuracy                           0.84       134
   macro avg       0.87      0.80      0.82       134
weighted avg       0.85      0.84      0.84       134



In [19]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_class_prior,param_fit_prior,param_force_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.016862,0.002832,0.001816,0.002732,1,None,True,True,"{'alpha': 1, 'class_prior': None, 'fit_prior':...",0.726566,0.776838,0.67375,0.818941,NaN,NaN,NaN,1


In [20]:
from sklearn.metrics import f1_score
f1_weighted=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_weighted value for best parameter {}:".format(grid.best_params_),f1_weighted)

The f1_weighted value for best parameter {'alpha': 1, 'class_prior': None, 'fit_prior': True, 'force_alpha': True}: 0.8354758282828774


In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8955582232893157